In [1]:
from azureml.core.workspace import Workspace

from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.conda_dependencies import CondaDependencies


ws = Workspace.from_config()
model = Model(ws, name="RFHD-AutoPrices", version=1)

In [9]:



# https://stackoverflow.com/questions/61803031/azure-ml-include-additional-files-during-model-deployment
# Disabled since the rubric implies a specification 'file' is required, suggesting a custom yaml file
# myenv_inference = Environment(workspace=ws, name="AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference") 
mysklearn_env = Environment.from_conda_specification(name='mysklearn-env', file_path='myenv_dependencies_hdrive.yml')
# from azureml.core import Image
# build = mysklearn_env.build(workspace=ws)
# build.wait_for_completion(show_output=True)

# Combine scoring script & environment in Inference configuration
dummy_inference_config = InferenceConfig(entry_script="score_hdrive.py", environment=mysklearn_env, source_directory='./')

# Set deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 3, auth_enabled=True)

ws_name = "svc-autoprices-hdrive-2"
service = Model.deploy(
    ws,
    ws_name,
    [model],
    inference_config = dummy_inference_config,
    deployment_config = deployment_config,
    overwrite=True
)
# service.wait_for_deployment(show_output=True)

No Python version provided, defaulting to "3.6.2"


In [3]:
from azureml.core.workspace import Workspace


print(ws.webservices)

# Choose the webservice you are interested in

from azureml.core import Webservice

service = Webservice(ws, 'svc-autoprices-hdrive-2')

# service.wait_for_deployment(show_output=True)
print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)
primary, secondary = service.get_keys()
print(primary)
#Enable app insights
service.update(enable_app_insights=True)

{'svc-autoprices-hdrive-2': AciWebservice(workspace=Workspace.create(name='quick-starts-ws-160132', subscription_id='81cefad3-d2c9-4f77-a466-99a7f541c7bb', resource_group='aml-quickstarts-160132'), name=svc-autoprices-hdrive-2, image_id=None, compute_type=None, state=ACI, scoring_uri=None, tags=http://ea8e6155-0dc1-4bfe-91cd-12da646bcbef.southcentralus.azurecontainer.io/score, properties={}, created_by={'hasInferenceSchema': 'False', 'hasHttps': 'False'}), 'svc-autoprices-hdrive': AciWebservice(workspace=Workspace.create(name='quick-starts-ws-160132', subscription_id='81cefad3-d2c9-4f77-a466-99a7f541c7bb', resource_group='aml-quickstarts-160132'), name=svc-autoprices-hdrive, image_id=None, compute_type=None, state=ACI, scoring_uri=None, tags=http://82c9edab-de4e-4a8d-842c-21afd36ab1f9.southcentralus.azurecontainer.io/score, properties={}, created_by={'hasInferenceSchema': 'False', 'hasHttps': 'False'})}
Healthy
http://ea8e6155-0dc1-4bfe-91cd-12da646bcbef.southcentralus.azurecontainer.i

In [22]:

print(service.get_logs())

2021-10-06T06:25:13,733143400+00:00 - rsyslog/run 
2021-10-06T06:25:13,740947300+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-10-06T06:25:13,749433200+00:00 - nginx/run 
2021-10-06T06:25:13,756637500+00:00 - iot-server/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-10-06T06:25:14,115951900+00:00 - iot-server/finish 1 0
2021-10-06T06:25:14,119809900+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (70)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 100
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2021-10-06 06:25:17,763 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2021-10-06 06:25:17,765 | root | INFO | Starting up request id generator
2021-10-06 06:25:17,765 | root | INFO | Sta

In [4]:
import json
import requests
# service = Webservice(workspace=ws, name='svc-autoprices-hdrive')



# URL for the web service
scoring_uri = service.scoring_uri
# If the service is authenticated, set the key or token
key = primary


data = {
    "data":
    [
        {
            'miles': "51000",
            'year': "2020",
            'make': "Ford",
            'model': "F-150",
            'engine_size': "5",
            'body_type': "Pickup",
            'vehicle_type': "Truck",
            'drivetrain': "4WD",
            'transmission': "Automatic",
            'fuel_type': "Gas",
            'state': "ON"
            
        }
    ,
        {
            'miles': "58900",
            'year': "2018",
            'make': "Toyota",
            'model': "Corolla",
            'engine_size': "1.8",
            'body_type': "Sedan",
            'vehicle_type': "Car",
            'drivetrain': "FWD",
            'transmission': "Automatic",
            'fuel_type': "Gas",
            'state': "AB"
            
        }

    ]
}
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

[101326.01333333332, 102209.46]
